In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

** load data**

In [ ]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [ ]:
train.shape

In [ ]:
train.describe()

here, we can see -1 value. this data mean na values. so in EDA replace -1 to na values

In [ ]:
train = pd.read_csv('../input/train.csv', na_values=['-1', '-1.0'])
test = pd.read_csv('../input/test.csv', na_values=['-1', '-1.0'])

In [ ]:
train.head()

In [ ]:
train.info()

what we can see here is that features are anonymous

however, the varialbes such as cat, calc, bin we can see that mean(binary, category etc)

In [ ]:
train.target.unique()

In [ ]:
train.target.value_counts().plot(kind='bar')

** this data is very very very~~~~ unbalanced! **

afterwords, we have to go through the process of upsampling and downsampling.

# visualization

In [ ]:
test['target'] = np.nan
df = pd.concat([train, test], axis=0)

In [ ]:
def bar_plot(col, data, hue=None):
    f, ax = plt.subplots(figsize=(12, 5))
    sns.countplot(x = col, hue=hue, data = data)
    plt.show()

def dist_plot(col, data):
    f, ax = plt.subplots(figsize=(12, 5))
    sns.distplot(data[col].dropna(), kde=False, bins=10)
    plt.show()
    
def bar_plot_ci(col, data):
    f, ax = plt.subplots(figsize=(12, 5))
    sns.barplot(col, 'target', data=data)
    plt.show()

In [ ]:
#binary variables
binary = ['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
          'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_calc_15_bin', 
          'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']
# categorical variables
category = ['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 
            'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 
            'ps_car_10_cat', 'ps_car_11_cat']
# integer varialbes
integer = ['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06', 
           'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 
           'ps_calc_14', 'ps_car_11']
# float variables
floats = ['ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_car_12', 'ps_car_13',
          'ps_car_14', 'ps_car_15']

# binary variable

In [ ]:
for col in binary:
    bar_plot(col, df)

some data have very many zeros, and some data have very many 1(ps_ind_16_bins, ps_calc_16_bin, and 17)

# category variable

In [ ]:
for col in category:
    bar_plot(col, df)

ps_ind_02_cat : almost data in 1.0

ps_ind_05_cat : almost data in 0.0

ps_car_01_cat : almost data in 11.0, 7.0.

ps_car_11_cat : has many values

# integer variable

In [ ]:
for col in integer:
    bar_plot(col, df)

here you can see some interesting data that is gathered on one side

# float

In [ ]:
for col in floats:
    dist_plot(col, df)

In [ ]:
for col in floats:
    dist_plot(col, df)

# corr with heatmap

In [ ]:
corr = df.corr()
f, ax = plt.subplots(figsize=(15, 7))
sns.heatmap(corr, cmap='summer')

# with target variables

# binary vs target

In [ ]:
for col in binary:
    bar_plot_ci(col, df)

# category vs target


In [ ]:
for col in category:
    bar_plot_ci(col, df)

# integer vs target


In [ ]:
for col in integer:
    bar_plot_ci(col, df)

- ps_calc_18_bin is difficult to find a significant difference in the average target ratio. This shows low predictive ability as a variable.

- The 104 eigenvalues of ps_car_11_cat show a target ratio of 0.02 to 0.08. an indication of some predictability

- And in the case of calc, it doesn't seem very useful, in fact, the top kernels have eliminated all the variables associated with the calc.

# let's see unbalanced data


In [ ]:
train.target.value_counts().plot(kind='bar')

if this is not balanced, accuracy should not be used. because the accuracy is always high. even if you just set it to 0 it's more than 90 percent.

whall we test it?

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
labels = train.columns[2:]

X = train[labels]
y = train['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print('accuracy : %.2f' %(accuracy * 100))

In [ ]:
model = XGBClassifier()
model.fit(X_train[['ps_calc_01']], y_train)
y_pred = model.predict(X_test[['ps_calc_01']])

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(y_true=y_test, y_pred = y_pred)
print(conf_mat)

In [ ]:
labels = ['Class 0', 'Class 1']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
fig.colorbar(cax)


it doesn't fit very well. that's why we need resampling process

In [ ]:
count_class_0, count_class_1 = train.target.value_counts()

In [ ]:
df_class_0 = train[train['target'] == 0]
df_class_1 = train[train['target'] == 1]

In [ ]:
print(count_class_1)
print(count_class_0)

# random under-sampling




In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
df_class_0_under.head()

In [ ]:
df_class_0_under.shape

In [ ]:
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

In [ ]:
print(df_test_under.shape)
print(df_test_under.target.value_counts())

In [ ]:
df_test_under.target.value_counts().plot(kind="bar")

# random over-sampling

In [ ]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True)

In [ ]:
df_class_1_over.shape

In [ ]:
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.target.value_counts())

df_test_over.target.value_counts().plot(kind='bar', title='Count (target)');

you can get it this way!